In [64]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [65]:
train_data_dir = "/Users/deepanshudubb/Documents/accident/training"
val_data_dir = "/Users/deepanshudubb/Documents/accident/validation"

In [66]:
img_width, img_height = 150, 150

In [67]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [68]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary', 
    shuffle=False 
)

Found 1840 images belonging to 2 classes.


In [69]:
validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 460 images belonging to 2 classes.


In [70]:
def extract_features(generator):
    features = []
    labels = []
    for _ in range(generator.samples // generator.batch_size):
        batch_x, batch_y = next(generator)
        features.append(batch_x.reshape(batch_x.shape[0], -1))
        labels.append(batch_y)
    return np.concatenate(features), np.concatenate(labels)

In [71]:
train_features_knn, train_labels_knn = extract_features(train_generator)
val_features_knn, val_labels_knn = extract_features(validation_generator)


In [72]:

knn = KNeighborsClassifier(n_neighbors=5) 
knn.fit(train_features_knn, train_labels_knn)

KNeighborsClassifier()

In [73]:
knn_predictions = knn.predict(val_features_knn)
knn_accuracy = accuracy_score(val_labels_knn, knn_predictions)
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.609375


In [74]:
model_cnn = Sequential()

In [75]:

model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model_cnn.add(BatchNormalization()) 
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Flatten())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(1, activation='sigmoid'))


optimizer = Adam(learning_rate=0.0001) 
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

epochs = 20

model_cnn.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)

/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 355ms/step - accuracy: 0.5354 - loss: 2.6103 - val_accuracy: 0.5223 - val_loss: 0.7617
Epoch 2/20
 1/57 ━━━━━━━━━━━━━━━━━━━━ 17s 319ms/step - accuracy: 0.7812 - loss: 0.6786

/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7812 - loss: 0.6786 - val_accuracy: 0.5223 - val_loss: 0.7696
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 360ms/step - accuracy: 0.6578 - loss: 0.6038 - val_accuracy: 0.6585 - val_loss: 0.6397
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8750 - loss: 0.4872 - val_accuracy: 0.6406 - val_loss: 0.6476
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 23s 395ms/step - accuracy: 0.7054 - loss: 0.5844 - val_accuracy: 0.6094 - val_loss: 0.7864
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6875 - loss: 0.5763 - val_accuracy: 0.6094 - val_loss: 0.7806
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 358ms/step - accuracy: 0.6644 - loss: 0.5818 - val_accuracy: 0.6540 - val_loss: 0.7324
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8750 - loss: 0.4185 - val_accuracy: 0.6496 - val_loss: 0.7378
Epoch 9/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 362ms/step - accuracy: 0.6958 - loss: 0.5943 - val_accuracy: 0.6875 - val_

In [76]:
optimizer = Adam(learning_rate=0.0001)
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [77]:
epochs = 50 

model_cnn.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 24s 410ms/step - accuracy: 0.7784 - loss: 0.4706 - val_accuracy: 0.7634 - val_loss: 0.5323
Epoch 2/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9062 - loss: 0.2365 - val_accuracy: 0.7589 - val_loss: 0.5341
Epoch 3/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 22s 391ms/step - accuracy: 0.8179 - loss: 0.4160 - val_accuracy: 0.7656 - val_loss: 0.5031
Epoch 4/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9062 - loss: 0.2432 - val_accuracy: 0.7612 - val_loss: 0.5200
Epoch 5/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 23s 393ms/step - accuracy: 0.8252 - loss: 0.3761 - val_accuracy: 0.7411 - val_loss: 0.5444
Epoch 6/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9062 - loss: 0.2715 - val_accuracy: 0.7344 - val_loss: 0.5565
Epoch 7/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 22s 391ms/step - accuracy: 0.8790 - loss: 0.2962 - val_accuracy: 0.7388 - val_loss: 0.5303
Epoch 8/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8125 - loss: 0.4231 - val_accuracy: 0.

In [84]:
loss, accuracy = model_cnn.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)

print("CNN Accuracy:", accuracy)

14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 0.8527 - loss: 0.4576
CNN Accuracy: 0.7767857313156128


In [85]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array 
import numpy as np

In [111]:

def predict_accident(image_path, model):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255. 

    prediction = model.predict(img)
    if prediction[0][0] > 0.5:
        return "Noxxn-Accidental"
    else:
        return "Accidental"

In [112]:
test_image_path = "/Users/deepanshudubb/Documents/accident/fine.jpg"
result_cnn = predict_accident(test_image_path, model_cnn)
print("CNN Prediction:", result_cnn)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
CNN Prediction: Noxxn-Accidental


In [115]:
model_cnn.save("cnn_model.h5")

In [116]:
import pickle
with open("knn_model.pkl", "wb") as f:
    pickle.dump(knn, f) 